# Workshop Container
This notebook don't have any scientific interest in itself.

Its purpose is to reproduce a workflow-like structure using containerized tools.

If you didn't touch the definition file (*.def) **you should not be able to run the last step.**

---------

1 - We will first compute the total energy of a non-optimized caffeine molecule using the Self Consistent Field (SCF) method. [Explaination](https://pyscf.org/user/scf.html)

2 - We will then use the code xtb (extended tigh-binding) a semi-empirical computation code, an in between ab-initio and classical molecular dynamics code. [xtb doc](https://xtb-docs.readthedocs.io/en/latest/) To optimize our molecule's geometry.

3 - Then we'll compute the total energy again to see if there is any difference.

4 - Finally, we'd like to visualize our molecules, using the [ase](https://ase-lib.org/) library. **This step will need some edit in the definition file.**


## **0** - Setting up environment

In [ ]:
# Import needed python libraries
import os
from pyscf import gto, scf
from pyscf.geomopt.geometric_solver import optimize

In [ ]:
# Set python variables
molecule = "Water"
unopt_file = f"randomized-molecules/{molecule}_unoptimized.xyz"
opt_file = f"optimized-molecules/{molecule}_optimized.xyz"

# Set environment variables, used for non-python cells (i.e : ones calling xtb or edition work directory)
os.environ["UNOPT_FILE"] = unopt_file
os.environ["OPT_FILE"] = opt_file

## **1** - Compute SCF energy of unoptimized Caffeine molecule

In [ ]:
unopt = gto.M(atom=unopt_file).build()        # Convert an .xyz file to a PySCF Molecule Object
mf_unopt = scf.RHF(unopt).run()               # Compute SCF 
print(f"Total Energy = {mf_unopt.e_tot} Ha")

## **2** - Run geometry optimization with xtb

In [ ]:
!xtb $UNOPT_FILE --opt 

In [ ]:
# Clean work directory
!mkdir -p optimized-molecules
!rm charges wbo xtbopt.log xtbrestart xtbtopo.mol
!mv xtbopt.xyz $OPT_FILE

## **3** - Compute energy of optimized molecules

In [ ]:
opt = gto.M(atom=opt_file).build()
mf_opt = scf.RHF(opt).run()
print(f"Total Energy = {mf_opt.e_tot} Ha")
print(f"Diff optimized - non-optimized = {mf_opt.e_tot - mf_unopt.e_tot} Ha")

## **4** - Display Molecules

In [ ]:
# import ASE library
from ase.io import read
from ase.visualize import view

In [ ]:
unopt_view = read(unopt_file)
opt_view = read(opt_file)

In [ ]:
view(unopt_view, viewer="x3d") # display unoptimized molecule

In [ ]:
view(opt_view, viewer="x3d") # same but with the optimized one